In [32]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [26]:
# Function to load images and annotations
def load_data(image_folder, annotation_folder):
    image_files = os.listdir(image_folder)
    images = []
    annotations = []
    
    for img_file in image_files:
        img_path = os.path.join(image_folder, img_file)
        annotation_path = os.path.join(annotation_folder, img_file.replace('.jpg', '.xml'))
        
        # Load image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize image to 224x224
        images.append(img)
        
        # Load XML annotation
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        
        # Extract polygon points
        points = []
        for pt in root.findall('.//pt'):
            x = int(pt.find('x').text)
            y = int(pt.find('y').text)
            points.append((x, y))
        annotations.append(points)
    
    return np.array(images), annotations

# Load data
image_folder = 'Annonated_Shapes'
annotation_folder = 'Annonated_XML'
images, annotations = load_data(image_folder, annotation_folder)

In [27]:
# Convert annotations to binary edge images
def annotations_to_edges(annotations, image_shape):
    edge_images = []
    for annotation in annotations:
        edge_img = np.zeros(image_shape[:2], dtype=np.uint8)
        for i in range(len(annotation)):
            cv2.line(edge_img, annotation[i], annotation[(i+1)%len(annotation)], 255, 1)
        edge_images.append(edge_img)
    return np.array(edge_images)

edge_images = annotations_to_edges(annotations, images[0].shape)

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
train_images, val_images, train_edges, val_edges = train_test_split(images, edge_images, test_size=0.2, random_state=42)

# Normalize images
train_images = train_images.astype('float32') / 255.0
val_images = val_images.astype('float32') / 255.0
train_edges = train_edges.astype('float32') / 255.0
val_edges = val_edges.astype('float32') / 255.0

In [39]:
def build_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(input_shape[0] * input_shape[1], activation='sigmoid'),  # Assuming flat output for edge image
        layers.Reshape((input_shape[0], input_shape[1]))
    ])
    return model

input_shape = (224, 224, 3)
model = build_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 flatten_6 (Flatten)         (None, 346112)           

In [41]:
# 3. Training

model.fit(train_images, train_edges, epochs=20, batch_size=32, validation_data=(val_images, val_edges))

Epoch 1/20
13/13 [==============================] - 42s 3s/step - loss: 0.0747 - accuracy: 0.2760 - val_loss: 0.0788 - val_accuracy: 0.3296
Epoch 2/20
13/13 [==============================] - 42s 3s/step - loss: 0.0749 - accuracy: 0.3643 - val_loss: 0.0794 - val_accuracy: 0.3550
Epoch 3/20
13/13 [==============================] - 42s 3s/step - loss: 0.0753 - accuracy: 0.3757 - val_loss: 0.0800 - val_accuracy: 0.3580
Epoch 4/20
13/13 [==============================] - 42s 3s/step - loss: 0.0759 - accuracy: 0.3788 - val_loss: 0.0806 - val_accuracy: 0.3591
Epoch 5/20
13/13 [==============================] - 42s 3s/step - loss: 0.0760 - accuracy: 0.3802 - val_loss: 0.0806 - val_accuracy: 0.3598
Epoch 6/20
13/13 [==============================] - 42s 3s/step - loss: 0.0760 - accuracy: 0.3806 - val_loss: 0.0806 - val_accuracy: 0.3598
Epoch 7/20
13/13 [==============================] - 42s 3s/step - loss: 0.0760 - accuracy: 0.3806 - val_loss: 0.0806 - val_accuracy: 0.3599
Epoch 8/20
13/13 [==

In [42]:
# 4. Inference

def annotate_edge(image_path, model):
    # Load test image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img_norm = img.astype('float32') / 255.0
    img_norm = np.expand_dims(img_norm, axis=0)
    
    # Predict edge map
    edge_map = model.predict(img_norm)[0]
    edge_map = (edge_map > 0.5).astype(np.uint8) * 255
    
    # Convert edge map to XML annotation
    edge_points = np.column_stack(np.where(edge_map > 0))
    
    root = ET.Element("annotation")
    filename = ET.SubElement(root, "filename")
    filename.text = os.path.basename(image_path)
    
    obj = ET.SubElement(root, "object")
    name = ET.SubElement(obj, "name")
    name.text = "shape"
    
    polygon = ET.SubElement(obj, "polygon")
    for point in edge_points:
        pt = ET.SubElement(polygon, "pt")
        x = ET.SubElement(pt, "x")
        x.text = str(point[1])
        y = ET.SubElement(pt, "y")
        y.text = str(point[0])
    
    tree = ET.ElementTree(root)
    xml_path = image_path.replace('.jpg', '_annotated.xml')
    tree.write(xml_path)
    
    # Save annotated image
    contours, _ = cv2.findContours(edge_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
    annotated_img_path = image_path.replace('.jpg', '_annotated.jpg')
    cv2.imwrite(annotated_img_path, img)

# Test annotation on a sample image
test_image_path = 'img8.jpg'
annotate_edge(test_image_path, model)

1/1 [==============================] - 1s 1s/step
